## 1. Imports and Model Loading

In [20]:
import os

os.chdir("sam-3d-objects/notebook")

In [13]:
os.getcwd()

'/d/hpc/home/jn16867'

In [21]:
import uuid
import imageio
import numpy as np
from IPython.display import Image as ImageDisplay

PATH = os.getcwd()
TAG = "hf"

## 2. Load input image to lift to 3D (multiple objects)

In [23]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [4]:
print("CUDA available:", torch.cuda.is_available())
print("CUDA devices:", torch.cuda.device_count())

CUDA available: True
CUDA devices: 2


In [24]:
os.environ["LIDRA_SKIP_INIT"] = "true"

from pathlib import Path
from sam3d_objects.pipeline.inference_with_embeddings import InferenceWithEmbeddings
from sam3d_objects.data.precompute_embeddings import preprocess_sample


config_path = f"{PATH}/../checkpoints/{TAG}/pipeline.yaml"

data_dir = Path("../../3d-counting/data/Stacks-3D-Real/scenes")

pipeline = InferenceWithEmbeddings(config_path, compile=False)

2026-02-05 14:29:54.370 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
/d/hpc/home/jn16867/.conda/envs/sam3d-objects/lib/python3.11/site-packages/moge/model/v1.py:171: UserWarning: The following deprecated/invalid arguments are ignored: {'output_mask': True, 'split_head': True}
  warnings.warn(f"The following deprecated/invalid arguments are ignored: {deprecated_kwargs}")
2026-02-05 14:30:41.724 | WARNING  | sam3d_objects.data.dataset.tdfy.preprocessor:__post_init__:51 - No rgb pointmap normalizer provided, using scale + shift 
2026-02-05 14:30:41.726 | INFO     | sam3d_objects.pipeline.inference_pipeline:__init__:100 - self.device: cuda
2026-02-05 14:30:41.727 | INFO     | sam3d_objects.pipeline.inference_pipeline:__init__:101 - CUDA_VISIBLE_DEVICES: 0,1
2026-02-05 14:30:41.728 | INFO     | sam3d_objects.pipeline.inference_pipeline:__init__:102 - Actually using GPU: 0
2026-02-05 14:30:41.728 | INFO

In [25]:
%load_ext autoreload
%autoreload 2

In [26]:
!nvidia-smi

Thu Feb  5 14:36:00 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.126.09             Driver Version: 580.126.09     CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100S-PCIE-32GB          On  |   00000000:41:00.0 Off |                    0 |
| N/A   31C    P0             39W /  250W |   13520MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [27]:
from importlib import reload
import sam3d_objects.data.precompute_embeddings as pe

reload(pe)

<module 'sam3d_objects.data.precompute_embeddings' from '/d/hpc/home/jn16867/cso/sam-3d-objects/sam3d_objects/data/precompute_embeddings.py'>

In [48]:
for category_dir in sorted(data_dir.iterdir()):
    if (category_dir / "geco2_mask" / "image.png").exists():
        print(f"Processing sample for category: {category_dir}")
        pe.preprocess_sample(category_dir, pipeline)
    else:
        print(f"No GeCo2 mask found for category: {category_dir}")

2026-02-03 23:18:05.629 | INFO     | sam3d_objects.pipeline.inference_pipeline:merge_image_and_mask:584 - Replacing alpha channel with the provided mask
2026-02-03 23:18:05.726 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running condition embedder ...
2026-02-03 23:18:05.792 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embedder finishes!


Processing sample for category: ../../3d-counting/data/Stacks-3D-Real/scenes/beads
(434, 578, 3)
[  0 255]
[  0 255]
Computing embeddings for container


2026-02-03 23:18:08.412 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running condition embedder ...
2026-02-03 23:18:08.474 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embedder finishes!
2026-02-03 23:18:10.795 | INFO     | sam3d_objects.pipeline.inference_pipeline:decode_slat:606 - Decoding sparse latent...
2026-02-03 23:18:10.846 | INFO     | sam3d_objects.pipeline.inference_pipeline:postprocess_slat_output:540 - Postprocessing mesh with option with_mesh_postprocess False, with_texture_baking False...
2026-02-03 23:18:10.848 | INFO     | sam3d_objects.pipeline.inference_pipeline:merge_image_and_mask:584 - Replacing alpha channel with the provided mask
2026-02-03 23:18:10.932 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running condition embedder ...
2026-02-03 23:18:10.998 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embe

Computing embeddings for object


2026-02-03 23:18:13.624 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running condition embedder ...
2026-02-03 23:18:13.686 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embedder finishes!
2026-02-03 23:18:16.040 | INFO     | sam3d_objects.pipeline.inference_pipeline:decode_slat:606 - Decoding sparse latent...
2026-02-03 23:18:16.108 | INFO     | sam3d_objects.pipeline.inference_pipeline:postprocess_slat_output:540 - Postprocessing mesh with option with_mesh_postprocess False, with_texture_baking False...


...

[  0 255]
Computing embeddings for container


2026-02-03 23:26:09.070 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embedder finishes!
2026-02-03 23:26:11.691 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running condition embedder ...
2026-02-03 23:26:11.752 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embedder finishes!
2026-02-03 23:26:14.130 | INFO     | sam3d_objects.pipeline.inference_pipeline:decode_slat:606 - Decoding sparse latent...
2026-02-03 23:26:14.210 | INFO     | sam3d_objects.pipeline.inference_pipeline:postprocess_slat_output:540 - Postprocessing mesh with option with_mesh_postprocess False, with_texture_baking False...
2026-02-03 23:26:14.217 | INFO     | sam3d_objects.pipeline.inference_pipeline:merge_image_and_mask:584 - Replacing alpha channel with the provided mask
2026-02-03 23:26:14.326 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running conditio

Computing embeddings for object


2026-02-03 23:26:17.017 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:633 - Running condition embedder ...
2026-02-03 23:26:17.078 | INFO     | sam3d_objects.pipeline.inference_pipeline:get_condition_input:637 - Condition embedder finishes!
2026-02-03 23:26:19.453 | INFO     | sam3d_objects.pipeline.inference_pipeline:decode_slat:606 - Decoding sparse latent...
2026-02-03 23:26:19.526 | INFO     | sam3d_objects.pipeline.inference_pipeline:postprocess_slat_output:540 - Postprocessing mesh with option with_mesh_postprocess False, with_texture_baking False...


Saved embeddings for semicrosses3


In [28]:
import torch
from torch.utils.data import DataLoader
from sam3d_objects.data.count_dataset import CountDataset


dataset = CountDataset(data_dir)
print(f"Dataset length: {len(dataset)}")

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(
    dataset, [train_size, val_size]
)

train_loader = DataLoader(
    train_dataset,
    batch_size=1,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=lambda x: x[0]
)

val_loader = DataLoader(
    val_dataset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=lambda x: x[0]
)

Dataset length: 43


In [ ]:
from sam3d_objects.pipeline.sam3d_count_predictor import SAM3DCountPredictor
import torch.nn as nn

device = "cuda"
num_epochs = 100
lr = 1e-3
output_dir = Path("../model_checkpoints/stacks-3d/")

use_hybrid = False
model = SAM3DCountPredictor(use_hybrid=use_hybrid).to(device)
    
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, num_epochs)

mse_loss = nn.MSELoss()
l1_loss = nn.L1Loss()

best_val_mae = float('inf')

In [ ]:
# Load best current model and continue training
checkpoint = torch.load(Path("../model_checkpoints/stacks-3d/checkpoint_epoch_100.pth"), map_location=device)

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])

start_epoch = checkpoint["epoch"] + 1
best_val_mae = checkpoint["best_val_mae"]

In [ ]:
from tqdm import tqdm
from sam3d_objects.pipeline.inference_with_embeddings import extract_geometric_features, compute_geometric_count_estimate


for epoch in range(start_epoch, start_epoch + num_epochs):
    model.train()
    train_loss = 0
    train_mae = 0
    
    print(f"\nEpoch {epoch + 1}/{num_epochs} - Training...")
    
    for sample in tqdm(train_loader):
        container_out = sample['container_outputs']
        object_out = sample['object_outputs']
        true_count = torch.tensor([sample['true_count']], dtype=torch.float32).to(device)
        
        shape_latent_container = container_out['shape_latent'].to(device)
        shape_latent_object = object_out['shape_latent'].to(device)
            
        slat_features_container = container_out['slat_features'].to(device)
        slat_features_object = object_out['slat_features'].to(device)
        
        if slat_features_container.dim() == 2:
            slat_features_container = slat_features_container.unsqueeze(0)
        if slat_features_object.dim() == 2:
            slat_features_object = slat_features_object.unsqueeze(0)
        
        geom_feat = extract_geometric_features(container_out, object_out).unsqueeze(0).to(device)
        
        geometric_estimate = torch.tensor(
            [compute_geometric_count_estimate(container_out, object_out, 1.0)],
            dtype=torch.float32
        ).to(device)
        
        if use_hybrid:
            pred_count, correction = model(
                shape_latent_container,
                shape_latent_object,
                slat_features_container,
                slat_features_object,
                geom_feat,
                geometric_estimate
            )
        else:
            pred_count, _ = model(
                shape_latent_container,
                shape_latent_object,
                slat_features_container,
                slat_features_object,
                geom_feat,
                None
            )
            
        loss = 0.5 * mse_loss(pred_count, true_count) + 0.5 * l1_loss(pred_count, true_count)
        
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        
        train_loss += loss.item()
        train_mae += torch.abs(pred_count - true_count).item()
        
    print(f"Train loss at epoch {epoch + 1}: {train_loss}")
    print(f"Train mae at epoch {epoch + 1}: {train_mae}")
    
    scheduler.step()
    
    avg_train_loss = train_loss / len(train_dataset)
    avg_train_mae = train_mae / len(train_dataset)

    # Validation
    if epoch % 5 == 0 or epoch == num_epochs - 1:
        model.eval()
        val_loss = 0
        val_mae = 0
        print("Validating")
        with torch.no_grad():
            for sample in tqdm(val_loader):
                container_out = sample['container_outputs']
                object_out = sample['object_outputs']
                true_count = torch.tensor([sample['true_count']], dtype=torch.float32).to(device)
                
                shape_latent_container = container_out['shape_latent'].to(device)
                shape_latent_object = object_out['shape_latent'].to(device)
                    
                slat_features_container = container_out['slat_features'].to(device)
                slat_features_object = object_out['slat_features'].to(device)
                
                if slat_features_container.dim() == 2:
                    slat_features_container = slat_features_container.unsqueeze(0)
                if slat_features_object.dim() == 2:
                    slat_features_object = slat_features_object.unsqueeze(0)
                
                geom_feat = extract_geometric_features(container_out, object_out).unsqueeze(0).to(device)
                
                geometric_estimate = torch.tensor(
                    [compute_geometric_count_estimate(container_out, object_out, 1.0)],
                    dtype=torch.float32
                ).to(device)
                
                pred_count, _ = model(
                    shape_latent_container,
                    shape_latent_object,
                    slat_features_container,
                    slat_features_object,
                    geom_feat,
                    None
                )
                
                loss = 0.5 * mse_loss(pred_count, true_count) + 0.5 * l1_loss(pred_count, true_count)
                
                val_loss += loss
                val_mae += torch.abs(pred_count - true_count).item()
            
            avg_val_loss = val_loss / len(val_dataset)
            avg_val_mae = val_mae / len(val_dataset)
            
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"  Train Loss: {avg_train_loss:.4f}, Train MAE: {avg_train_mae:.2f}")
            print(f"  Val Loss: {avg_val_loss:.4f}, Val MAE: {avg_val_mae:.2f}")
            
            if avg_val_mae < best_val_mae:
                best_val_mae = avg_val_mae
                
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_val_mae': best_val_mae 
                }, output_dir / 'best_model.pth')
                print(f"  Saved new best model with VAL MAE: {avg_val_mae:.2f}")
            
            if (epoch + 1) % 10 == 0:
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, output_dir / f"checkpoint_epoch_{epoch+1}.pth")
        
print("\nTraining completed")
print(f"Best Validation MAE: {best_val_mae:.2f}")


Epoch 1/100 - Training...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:34<00:00,  1.02s/it]


Train loss at epoch 1: 215316283.37786865
Train mae at epoch 1: 38097.46598434448
Validating


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:11<00:00,  1.30s/it]


Epoch 1/100
  Train Loss: 6332831.8641, Train MAE: 1120.51
  Val Loss: 1785676.8750, Val MAE: 932.89
  Saved new best model with VAL MAE: 932.89

Epoch 2/100 - Training...


...

Train loss at epoch 99: 141857641.5042501
Train mae at epoch 99: 18700.07559776306

Epoch 100/100 - Training...


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:34<00:00,  1.02s/it]


Train loss at epoch 100: 150982033.54672563
Train mae at epoch 100: 19233.360382080078
Validating


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:20<00:00,  2.27s/it]


Epoch 100/100
  Train Loss: 4440648.0455, Train MAE: 565.69
  Val Loss: 1620652.5000, Val MAE: 882.20

Training completed
Best Validation MAE: 793.02
